# Foundations of Computer Science - progetto 2022/23

- Mattia Birti [897092] 
- Alberto Porrini [826306]
- Gloria Longo [864579]

You have to work on the Dogs adoptions dataset:
*  [Dogs](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogs.csv)
*  [Dog Travel](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/dogTravel.csv)
*  [NST-EST2021-POP](https://github.com/mat1218B/ProjectOfComputerScience/blob/main/adoptions/NST-EST2021-POP.csv)

### Notes

1. It is mandatory to use GitHub for developing the project.
1. The project must be a jupyter notebook.
1. There is no restriction on the libraries that can be used, nor on the Python version.
1. All questions on the project must be asked in a public channel on Zulip.
1. At most 3 students can be in each group. You must create the groups by yourself.
1. You do not have to send me the project before the discussion.s

## Libraries

In [1]:
import pandas as pd

### Import data from GitHub

#### Dogs.csv

In [2]:
dogs = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogs.csv", sep=",", encoding='latin-1')

In [3]:
dogs.shape

(58180, 37)

In [4]:
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


#### DogTravel.csv

In [5]:
dogTravel = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/dogTravel.csv", sep=",", encoding='latin-1')

In [6]:
dogTravel.shape

(6194, 9)

In [7]:
dogTravel.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


#### NST-EST.csv

In [8]:
nstest = pd.read_csv("https://raw.githubusercontent.com/mat1218B/ProjectOfComputerScience/main/adoptions/NST-EST2021-POP.csv", header=None, sep=",", encoding='latin-1')

In [9]:
nstest.shape

(51, 2)

In [10]:
nstest

,0,1
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223
5,Colorado,5.773.714
6,Connecticut,3.605.944
7,Delaware,989.948
8,District of Columbia,689.545
9,Florida,21.538.187


## 1. Extract all dogs with status that is not adoptable

## 2. For each (primary) breed, determine the number of dogs

In [11]:
dogs.groupby('breed_primary').count()['id']

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Name: id, Length: 216, dtype: int64

## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

Let's see all the primary breed with a groupBy:

In [12]:
nBreed = dogs.groupby('breed_primary').count()[['species','breed_secondary']]
nBreed

,species,breed_secondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


As we can see we can use "species" as total number of individues for this species.
Affenpischer are in total 17 dogs, 2 of those are mixed breed.

Rename colums

In [13]:
nBreed = nBreed.rename( columns = {'species':'total', 'breed_secondary':'nSecondary'})
nBreed

,total,nSecondary
breed_primary,,
Affenpinscher,17,2
Afghan Hound,4,1
Airedale Terrier,19,9
Akbash,3,0
Akita,181,52
...,...,...
Wirehaired Pointing Griffon,1,0
Wirehaired Terrier,60,18
Xoloitzcuintli / Mexican Hairless,11,4


We have to compute how many are primary breed:

In [14]:
nBreed['nPrimary'] = nBreed['total'] - nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary
breed_primary,,,
Affenpinscher,17,2,15
Afghan Hound,4,1,3
Airedale Terrier,19,9,10
Akbash,3,0,3
Akita,181,52,129
...,...,...,...
Wirehaired Pointing Griffon,1,0,1
Wirehaired Terrier,60,18,42
Xoloitzcuintli / Mexican Hairless,11,4,7


It's time to compute the ratio between Primary and Secondary Breed.

In [15]:
nBreed['ratioBreed'] = nBreed['nPrimary'] / nBreed['nSecondary']
nBreed

,total,nSecondary,nPrimary,ratioBreed
breed_primary,,,,
Affenpinscher,17,2,15,7.500000
Afghan Hound,4,1,3,3.000000
Airedale Terrier,19,9,10,1.111111
Akbash,3,0,3,inf
Akita,181,52,129,2.480769
...,...,...,...,...
Wirehaired Pointing Griffon,1,0,1,inf
Wirehaired Terrier,60,18,42,2.333333
Xoloitzcuintli / Mexican Hairless,11,4,7,1.750000


As we can see, there are many 'inf' values. It is correct because for that breed there are not any secondary breed.

## 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

## 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

Explore the dog's column names

In [16]:
dogs.columns

Index(['id', 'org_id', 'url', 'type.x', 'species', 'breed_primary',
       'breed_secondary', 'breed_mixed', 'breed_unknown', 'color_primary',
       'color_secondary', 'color_tertiary', 'age', 'sex', 'size', 'coat',
       'fixed', 'house_trained', 'declawed', 'special_needs', 'shots_current',
       'env_children', 'env_dogs', 'env_cats', 'name', 'status', 'posted',
       'contact_city', 'contact_state', 'contact_zip', 'contact_country',
       'stateQ', 'accessed', 'type.y', 'description', 'stay_duration',
       'stay_cost'],
      dtype='object')

Compute the mean of the stay duration and stay cost for the pair AGE, SIZE;

In [17]:
AgeSize = dogs.groupby(['age', 'size'])[['stay_duration','stay_cost']].mean()
AgeSize

stay_duration   stay_cost
age    size                                  
Adult  Extra Large      89.015414  232.591561
       Large            89.531943  238.661141
       Medium           89.421036  238.258977
       Small            89.407479  238.974838
Baby   Extra Large      87.032967  237.180879
       Large            89.701564  238.698827
       Medium           89.577668  237.108131
       Small            89.958291  239.083810
Senior Extra Large      88.861111  235.232361
       Large            88.984298  237.507364
       Medium           89.810052  238.514615
       Small            89.073626  238.282286
Young  Extra Large      90.586345  245.835582
       Large            90.104206  238.149506
       Medium           89.515123  239.304603
       Small            89.814275  241.540069

## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

## 9. For each state, compute the ratio between the number of travels and the population.

In [18]:
import pycountry #have to install it (pip install pycountry)

### prima analisi del dataset

In [19]:
dogTravel.shape #vedo il numero di viaggi totali effettuati

(6194, 9)

In [20]:
dogTravel.groupby(['contact_state']).count().shape 
#vedo il numero di paesi differenti che ha effettuato viaggi

(45, 8)

In [21]:
nstest.shape

(51, 2)

### so che nstest contiene 51stati, solo 45 stati hanno effettuato viaggi per un totale di 6149 viaggi

In [22]:
nTravelForState = dogTravel.groupby(['contact_state']).count()[['index']].reset_index().rename(columns={'contact_state':'state', 'index':'nTravel'})
nTravelForState.head()
#giusta la somma dei viaggi è 6194 e ha 45 stati

,state,nTravel
0,17325,10
1,AL,75
2,AR,10
3,AZ,70
4,CA,28


In [23]:
nTravelForState['nTravel'].sum()

6194

In [24]:
nTravelForState.shape

(45, 2)

### giustamente contiene 45 stati per un totale di 6149 viaggi. !!! c'è un dato sbagliato, andiamo a vedere a cosa fa riferimento

In [25]:
dogTravel[dogTravel['contact_state'] == '17325']

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
3237,3237,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,New York,NaN,True,NaN
3238,3238,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",New York,NaN,True,NaN
3714,3714,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Pennsylvania,NaN,True,NaN
3715,3715,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Pennsylvania,NaN,True,NaN
6029,6029,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Virginia,NaN,True,NaN
6030,6030,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Virginia,NaN,True,NaN
6074,6074,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,Washington DC,NaN,True,NaN
6075,6075,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",Washington DC,NaN,True,NaN
6133,6133,36978896,PA,17325,Maddie is our little Miss Cutie Patootie! She ...,West Virginia,NaN,True,NaN
6134,6134,33218331,PA,17325,"Born in August 2014, Bucky has a great sense o...",West Virginia,NaN,True,NaN


### lo zip fa riferimento allo stato della pensilvenia, salviamo i viaggi in una variabile e copiamoli all'interno del campo corretto

In [26]:
#controlliamo prima il valore della pensilvania
nTravelForState[nTravelForState['state']=='PA'] #316 quindi totale 326

,state,nTravel
34,PA,316


In [27]:
#mi salvo i valori
PAerr = nTravelForState[nTravelForState['state'] == '17325']['nTravel']
PAcorr = nTravelForState[nTravelForState['state'] == 'PA']['nTravel']

In [28]:
#elimino la riga con il valore errato
nTravelForState = nTravelForState.drop(nTravelForState.index[nTravelForState['state'] == '17325']).reset_index()

In [29]:
nTravelForState.head()

,index,state,nTravel
0,1,AL,75
1,2,AR,10
2,3,AZ,70
3,4,CA,28
4,5,CO,103


In [30]:
#sommo il valore ai viaggi di PA
nTravelForState.loc[nTravelForState['state']=='PA', 'nTravel'] = sum(PAcorr, PAerr).values
nTravelForState[nTravelForState['state'] == 'PA']

,index,state,nTravel
33,34,PA,326


In [31]:
#controllo che il numero di viaggi totali sia giusto
nTravelForState['nTravel'].sum()

6194

### abbiamo quindi la tabella corretta con gli stati e il numero di viaggi (numero di stati che viaggiano sono quindi 44)

### creiamo ad ok la tabella stati e popolazione

In [32]:
state = nstest.rename(columns = {0:'state', 1:'nPopulation'})
state

,state,nPopulation
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223
5,Colorado,5.773.714
6,Connecticut,3.605.944
7,Delaware,989.948
8,District of Columbia,689.545
9,Florida,21.538.187


### aggiungiamo le sigle ad ogni stato per fare il merge

In [33]:
def findCountryAlpha2 (country_name):
    try:
        sub = pycountry.subdivisions.lookup(country_name)
        sample_str = sub.code
        stateCode = sub.code[-2:]
        return stateCode
    except:
        return ("not founded!")

state['state_code'] = state.apply(lambda row: findCountryAlpha2(row.state) , axis = 1)
state.head()

,state,nPopulation,state_code
0,Alabama,5.024.279,AL
1,Alaska,733.391,AK
2,Arizona,7.151.502,AZ
3,Arkansas,3.011.524,AR
4,California,39.538.223,CA


### Dopo alcune prove ci accorgiamo che la libreria pycountry ha un dato sbagliato Montana != MT

In [34]:
#il montana  sbagliato nella libreria va corretto con MT state code non ha senso
state[state['state']=='Montana']

,state,nPopulation,state_code
26,Montana,1.084.225,12


In [35]:
state.loc[state['state']=='Montana', 'state_code'] = 'MT'
state[state['state']=='Montana']

,state,nPopulation,state_code
26,Montana,1.084.225,MT


### Altro errore della library: devo correggere in 'state' la sigla del Maryland in MY

In [36]:
state[state['state']=='Maryland']

,state,nPopulation,state_code
20,Maryland,6.177.224,MY


In [37]:
state.loc[state['state']=='Maryland', 'state_code'] = 'MD'
state[state['state']=='Maryland']

,state,nPopulation,state_code
20,Maryland,6.177.224,MD


### Errore nel csv: devo correggere in nTravelForState quando trovo NB metto NJ

In [38]:
nTravelForState[nTravelForState['state']=='NB']

,index,state,nTravel
23,24,NB,2


In [39]:
nTravelForState[nTravelForState['state']=='NJ']

,index,state,nTravel
26,27,NJ,552


### elimino la riga 23 e sommo il valore alla riga 26

In [40]:
#sommo i due valori
NJerr = nTravelForState[nTravelForState['state'] == 'NB']['nTravel']
NJcorr = nTravelForState[nTravelForState['state'] == 'NJ']['nTravel']

In [41]:
#elimino la riga con la sigla sbagliata
nTravelForState = nTravelForState.drop(nTravelForState.index[nTravelForState['state'] == 'NB']).reset_index()

In [42]:
#salvo il valore in NJ
nTravelForState.loc[nTravelForState['state']=='NJ', 'nTravel'] = sum(NJcorr, NJerr).values
nTravelForState[nTravelForState['state'] == 'NJ']

,level_0,index,state,nTravel
25,26,27,NJ,554


In [43]:
### tutti i dati ora sono corretti, momento di merging

In [44]:
table_travel_for_state = pd.merge(state, nTravelForState, left_on='state_code', right_on='state')
table_travel_for_state

,state_x,nPopulation,state_code,level_0,index,state_y,nTravel
0,Alabama,5.024.279,AL,0,1,AL,75
1,Arizona,7.151.502,AZ,2,3,AZ,70
2,Arkansas,3.011.524,AR,1,2,AR,10
3,California,39.538.223,CA,3,4,CA,28
4,Colorado,5.773.714,CO,4,5,CO,103
5,Connecticut,3.605.944,CT,5,6,CT,90
6,Delaware,989.948,DE,7,8,DE,57
7,District of Columbia,689.545,DC,6,7,DC,112
8,Florida,21.538.187,FL,8,9,FL,133
9,Georgia,10.711.908,GA,9,10,GA,109


In [45]:
table_travel_for_state['nTravel'].sum()

6194

#### proviamo a ricavare i zip degli stati per adesso

In [46]:
nTravelForState.shape

(43, 4)

In [47]:
state.shape

(51, 3)

In [48]:
table_travel_for_state.shape

(43, 7)

In [49]:
#abbiamo 51 stati americani 
#45 stati che viaggiano 
#ma dopo il merge ne abbiamo 42 
#c'è qualcoda che non va

#provo a fare nTravelForState - table_travel_for_state e vedo quali mi rimangono fuori
#il montana è sbagliato va corretto

In [50]:
prova = table_travel_for_state[table_travel_for_state["state_code"] == nTravelForState["contact_state"]]
prova

KeyError: 'contact_state'

## 10. For each dog, compute the number of days from the posted day to the day of last access.

## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.